# BursterCa 

A Ca-based burster.

In [ ]:
# Set the path
import sys
sys.path.append("../../")

from act.cell_model import ACTCellModel
from act.simulator import ACTSimulator
from act.types import SimulationParameters, ConstantCurrentInjection, FilterParameters, ConductanceOptions, OptimizationParameters
import act.data_processing as dp
from act.module import ACTModule

import numpy as np
import matplotlib.pyplot as plt

from act.passive import ACTPassiveModule
from act.segregation import ACTSegregator

from sklearn.metrics import mean_absolute_error
from act.metrics import pp_error

## 1. Set the "target" model and simulate from it

The "target" model's output represents the target bio data provided by the user to tune for.

Parameters:
...

In [ ]:
# Define the "target" cell
target_cell = ACTCellModel(
    cell_name = "BursterCa",
    path_to_hoc_file = "../../data/BursterCa/orig/target_template.hoc",
    path_to_mod_files = "../../data/BursterCa/orig/modfiles/",
    passive = ["gbar_leak", "eleak", None],
    active_channels = ["gbar_na", "gbar_kdr", "gbar_cas", "gbar_ka", "gbar_cat"]
)

In [ ]:
target_g = np.array([4e-05, 0.13, 0.1, 0.01, 0.17, 0.005])

### 1.1. Target passive properties

In [ ]:
# Set simulations
simulator = ACTSimulator(output_folder_name = "output")

sim_params = SimulationParameters(
    sim_name = "target",
    sim_idx = 0,
    h_celsius = 6.3,
    h_dt = 0.1,
    h_tstop = 1000,
    CI = [ConstantCurrentInjection(amp = -0.2, dur = 700, delay = 100)])

simulator.submit_job(target_cell, sim_params)
simulator.run_jobs(1)

In [ ]:
passive_trace = np.load("output/target/out_0.npy")[:, 0]
plt.plot(passive_trace[::10])

In [ ]:
target_gpp = ACTPassiveModule.compute_gpp(passive_trace, 0.1, 100, 700, -0.2)
target_gpp

### 1.2. Target FI curve

In [ ]:
# Set simulations
simulator = ACTSimulator(output_folder_name = "output")

for sim_idx, amp_value in enumerate([0.1, 1.0, 3.0]):
    sim_params = SimulationParameters(
        sim_name = "target",
        sim_idx = sim_idx,
        h_celsius = 6.3,
        h_dt = 0.1,
        h_tstop = 1000,
        CI = [ConstantCurrentInjection(amp = amp_value, dur = 700, delay = 100)])

    simulator.submit_job(target_cell, sim_params)

simulator.run_jobs(3)

# Combine simulated traces into one dataset for convenience
dp.combine_data("output/target")

In [ ]:
# Plot the traces and the FI curve
simulated_data = np.load("output/target/combined_out.npy") # 3 x 10000 x 4; (n_sim x time x [V, I, g, lto_hto])

fig, ax = plt.subplots(1, 3, figsize = (10, 2))
for axid, amp in enumerate([0.1, 1.0, 3.0]):
    ax[axid].plot(simulated_data[axid, ::10, 0])
    ax[axid].set_xlabel("Time (ms)")
    ax[axid].set_title(f"CI = {amp} nA")

ax[0].set_ylabel("Voltage (mV)")

In [ ]:
simulated_data = np.load("output/target/combined_out.npy")

f = []
for trace_id in range(len(simulated_data)):
    f.append(len(dp.find_events(simulated_data[trace_id, ::10, 0].flatten())))

plt.plot([0.1, 1.0, 3.0], f)
plt.xlabel("CI (nA)")
plt.ylabel("Frequency (Hz)")
plt.title("FI Curve")

## 2. Original pipeline - optimize passive and active channels together

### 2.1. Set the train cell 

We assume the train cell was acquired from an external source, e.g., AllenDB. Thus, its parameters do not necesserily match those of the target cell. Here we set these parameters to the target's values +- 10% std.

In [ ]:
random_state = np.random.RandomState(123)
target_values = np.array([1, 0.13, 0.1, 0.01, 0.17, 0.005, 4e-05, -50])
# Cm, gnabar, gbar_kdr, gbar_cas, gbar_ka, gbar_cat, gbar_leak, eleak
target_values = target_values + random_state.normal(0, np.abs(target_values * 0.1))
target_values

Train cell (updated in template.hoc):

- cm = 8.9e-1
- gbar_na = 1.42965e-1 (S/cm2)
- gbar_kdr = 1.02829e-1 (S/cm2)
- gbar_cas = 8.493705e-3 (S/cm2)
- gbar_ka = 1.601638e-1 (S/cm2)
- gbar_cat = 5.825718e-3 (S/cm2)
- gbar_leak = 3.029328e-5 (S/cm2)
- eleak = 52.14456 (mV)

In [ ]:
# Define the train cell
train_cell = ACTCellModel(
    cell_name = "BursterCa",
    path_to_hoc_file = "../../data/BursterCa/seg/template.hoc",
    path_to_mod_files = "../../data/BursterCa/seg/modfiles/",
    passive = ["eleak", None],
    active_channels = ["gbar_na", "gbar_kdr", "gbar_cas", "gbar_ka", "gbar_cat", "gbar_leak"],
)

### 2.2. Passive properties before tuning

In [ ]:
# Set simulations
simulator = ACTSimulator(output_folder_name = "output")

sim_params = SimulationParameters(
    sim_name = "seg_orig",
    sim_idx = 0,
    h_celsius = 6.3,
    h_dt = 0.1,
    h_tstop = 1000,
    CI = [ConstantCurrentInjection(amp = -0.2, dur = 700, delay = 100)])

simulator.submit_job(train_cell, sim_params)
simulator.run_jobs(1)

passive_trace = np.load("output/seg_orig/out_0.npy")[:, 0]
ACTPassiveModule.compute_gpp(passive_trace, 0.1, 100, 700, -0.2)

### 2.3. FI curve before tuning

In [ ]:
# Set simulations
simulator = ACTSimulator(output_folder_name = "output")

for sim_idx, amp_value in enumerate([0.1, 1.0, 3.0]):
    sim_params = SimulationParameters(
        sim_name = "seg_orig",
        sim_idx = sim_idx,
        h_celsius = 6.3,
        h_dt = 0.1,
        h_tstop = 1000,
        CI = [ConstantCurrentInjection(amp = amp_value, dur = 700, delay = 100)])

    simulator.submit_job(train_cell, sim_params)

simulator.run_jobs(3)

# Combine simulated traces into one dataset for convenience
dp.combine_data("output/seg_orig")

In [ ]:
simulated_data = np.load("output/seg_orig/combined_out.npy")

f = []
for trace_id in range(len(simulated_data)):
    f.append(len(dp.find_events(simulated_data[trace_id, ::10, 0].flatten())))

plt.plot([0.1, 1.0, 3.0], f)
plt.xlabel("CI (nA)")
plt.ylabel("Frequency (Hz)")
plt.title("FI Curve")

### 2.4 Optimize

In [ ]:
# Parameter ranges as if provided by the user

gbar_leak = 4e-05
gbar_na = 0.13
gbar_kdr = 0.1
gbar_cas = 0.01
gbar_ka = 0.17
gbar_cat = 0.005

random_state = np.random.RandomState(123)
gbar_leak_range = (gbar_leak - random_state.uniform(0, gbar_leak / 2), gbar_leak + random_state.uniform(0, gbar_leak / 2))
gbar_na_range = (gbar_na - random_state.uniform(0, gbar_na / 2), gbar_na + random_state.uniform(0, gbar_na / 2))
gbar_kdr_range = (gbar_kdr - random_state.uniform(0, gbar_kdr / 2), gbar_kdr + random_state.uniform(0, gbar_kdr / 2))
gbar_cas_range = (gbar_cas - random_state.uniform(0, gbar_cas / 2), gbar_cas + random_state.uniform(0, gbar_cas / 2))
gbar_ka_range = (gbar_ka - random_state.uniform(0, gbar_ka / 2), gbar_ka + random_state.uniform(0, gbar_ka / 2))
gbar_cat_range = (gbar_cat - random_state.uniform(0, gbar_cat / 2), gbar_cat + random_state.uniform(0, gbar_cat / 2))


print(f"Leak: {gbar_leak_range}")
print(f"Na: {gbar_na_range}")
print(f"Kdr: {gbar_kdr_range}")
print(f"Cas: {gbar_cas_range}")
print(f"Ka: {gbar_ka_range}")
print(f"Cat: {gbar_cat_range}")

In [ ]:
# Possibly adjsut
train_cell = ACTCellModel(
    cell_name = "BursterCa",
    path_to_hoc_file = "../../data/BursterCa/seg/template.hoc",
    path_to_mod_files = "../../data/BursterCa/seg/modfiles/",
    passive = ["eleak", None],
    active_channels = ["gbar_na", "gbar_kdr", "gbar_cas", "gbar_ka", "gbar_cat", "gbar_leak"]
)

In [ ]:
sim_params = SimulationParameters(
        sim_name = "cell",
        sim_idx = 0,
        h_celsius = 6.3,
        h_dt = 0.1,
        h_tstop = 1000)

optim_params = OptimizationParameters(
    conductance_options = [
        ConductanceOptions(variable_name = "gbar_na", low = gbar_na_range[0], high = gbar_na_range[1], n_slices = 3),
        ConductanceOptions(variable_name = "gbar_kdr", low = gbar_kdr_range[0], high = gbar_kdr_range[1], n_slices = 3),
        ConductanceOptions(variable_name = "gbar_cas", low = gbar_cas_range[0], high = gbar_cas_range[1], n_slices = 3),  
        ConductanceOptions(variable_name = "gbar_ka", low = gbar_ka_range[0], high = gbar_ka_range[1], n_slices = 3),
        ConductanceOptions(variable_name = "gbar_cat", low = gbar_cat_range[0], high = gbar_cat_range[1], n_slices = 3),
        ConductanceOptions(variable_name = "gbar_leak", low = gbar_leak_range[0], high = gbar_leak_range[1], n_slices = 3)
    ],
    CI_options = [
        ConstantCurrentInjection(amp = 0.1, dur = 700, delay = 100),
        ConstantCurrentInjection(amp = 1.0, dur = 700, delay = 100),
        ConstantCurrentInjection(amp = 3.0, dur = 700, delay = 100)
    ],
    filter_parameters = FilterParameters(
        saturation_threshold = -55,
        window_of_inspection = (100, 800)
    )
)

m = ACTModule(
    name = "seg_orig",
    cell = train_cell,
    simulation_parameters = sim_params,
    optimization_parameters = optim_params,
    target_file = "output/target/combined_out.npy"
)

In [ ]:
m.run()

In [ ]:
orig_g = np.array(list(m.cell.prediction.values()))

In [ ]:
# Test g error
mean_absolute_error(target_g, orig_g)

### 2.5. Passive properties after tuning

In [ ]:
train_cell = ACTCellModel(
    cell_name = "BursterCa",
    path_to_hoc_file = "../../data/BursterCa/seg/template.hoc",
    path_to_mod_files = "../../data/BursterCa/seg/modfiles/",
    passive = ["eleak", None],
    active_channels = ["gbar_na", "gbar_kdr", "gbar_cas", "gbar_ka", "gbar_cat", "gbar_leak"]
)

In [ ]:
# Set simulations
simulator = ACTSimulator(output_folder_name = "output")

sim_params = SimulationParameters(
    sim_name = "burster_seg_orig_after",
    sim_idx = 0,
    h_celsius = 6.3,
    h_dt = 0.1,
    h_tstop = 1000,
    CI = [ConstantCurrentInjection(amp = -0.2, dur = 700, delay = 100)])

train_cell.set_g_bar(["gbar_na", "gbar_kdr", "gbar_cas", "gbar_ka", "gbar_cat", "gbar_leak"], orig_g)

simulator.submit_job(train_cell, sim_params)
simulator.run_jobs(1)

passive_trace = np.load("output/burster_seg_orig_after/out_0.npy")[:, 0]
ACTPassiveModule.compute_gpp(passive_trace, 0.1, 100, 700, -0.2)

In [ ]:
orig_gpp = ACTPassiveModule.compute_gpp(passive_trace, 0.1, 100, 700, -0.2)
orig_gpp

In [ ]:
pp_error(target_gpp, orig_gpp)

### 2.6. FI curve after tuning

In [ ]:
train_cell = ACTCellModel(
    cell_name = "BursterCa",
    path_to_hoc_file = "../../data/BursterCa/seg/template.hoc",
    path_to_mod_files = "../../data/BursterCa/seg/modfiles/",
    passive = ["eleak", None],
    active_channels = ["gbar_na", "gbar_kdr", "gbar_cas", "gbar_ka", "gbar_cat", "gbar_leak"]
)

In [ ]:
# Set simulations
simulator = ACTSimulator(output_folder_name = "output")


for sim_idx, amp_value in enumerate([0.1, 1.0, 3.0]):
    sim_params = SimulationParameters(
        sim_name = "burster_seg_orig_after",
        sim_idx = sim_idx,
        h_celsius = 6.3,
        h_dt = 0.1,
        h_tstop = 1000,
        CI = [ConstantCurrentInjection(amp = amp_value, dur = 700, delay = 100)])
    
    train_cell.set_g_bar(
        ["gbar_na", "gbar_kdr", "gbar_cas", "gbar_ka", "gbar_cat", "gbar_leak"], 
        orig_g)


    simulator.submit_job(train_cell, sim_params)

simulator.run_jobs(3)

# Combine simulated traces into one dataset for convenience
dp.combine_data("output/burster_seg_orig_after")

In [ ]:
simulated_data = np.load("output/burster_seg_orig_after/combined_out.npy")

f = []
for trace_id in range(len(simulated_data)):
    f.append(len(dp.find_events(simulated_data[trace_id, ::10, 0].flatten())))

plt.plot([0.1, 1.0, 3.0], f)
plt.xlabel("CI (nA)")
plt.ylabel("Frequency (Hz)")
plt.title("FI Curve")